In [21]:
import pandas as pd
import numpy as np
import re
from pandas import Index
import matplotlib.pyplot as plt

In [54]:
parca_table =pd.read_csv("../data/airparca_20170808_20180809.csv",sep=";")

In [55]:
parca_table.head()

,Station,Polluant,Mesure,Unité,08/08/2017 0:00,08/08/2017 1:00,08/08/2017 2:00,08/08/2017 3:00,08/08/2017 4:00,08/08/2017 5:00,...,08/08/2018 15:00,08/08/2018 16:00,08/08/2018 17:00,08/08/2018 18:00,08/08/2018 19:00,08/08/2018 20:00,08/08/2018 21:00,08/08/2018 22:00,08/08/2018 23:00,09/08/2018 0:00
0,Manosque,Ozone (O3),Ozone,µg/m³,85,80,83,100,110,111,...,98,97,94,90,97,91,51,62,39,34
1,Manosque,Particules PM10,Particules PM10,µg/m³,13,10,8,16,13,14,...,19,16,20,16,15,17,17,19,15,15
2,Obs Haute-Provence,Ozone (O3),Ozone,µg/m³,104,109,109,106,107,112,...,103,101,102,107,98,96,89,88,77,73
3,Obs Haute-Provence,Particules PM10,Particules PM10,µg/m³,12,12,11,15,15,15,...,14,13,-,-,25,20,18,17,20,16
4,Obs Haute-Provence,"Particules PM2,5",Particules PM2.5,µg/m³,6,10,9,8,10,15,...,10,11,14,11,8,8,10,9,11,10


In [56]:
location_pattern = re.compile("Nice|Marseille",re.IGNORECASE)

In [57]:
pollutant_bool = (parca_table.Mesure == 'Particules PM2.5') | (parca_table.Mesure == 'Ozone')

In [58]:
location_bool = list(map(lambda location: True if location_pattern.search(location) else False,parca_table.Station))

In [59]:
index_bool  = pollutant_bool & location_bool

In [60]:
parca_table=parca_table.loc[index_bool].drop(columns=["Polluant","Unité"])

In [61]:
parca_table.head()

,Station,Mesure,08/08/2017 0:00,08/08/2017 1:00,08/08/2017 2:00,08/08/2017 3:00,08/08/2017 4:00,08/08/2017 5:00,08/08/2017 6:00,08/08/2017 7:00,...,08/08/2018 15:00,08/08/2018 16:00,08/08/2018 17:00,08/08/2018 18:00,08/08/2018 19:00,08/08/2018 20:00,08/08/2018 21:00,08/08/2018 22:00,08/08/2018 23:00,09/08/2018 0:00
11,Nice_Port,Particules PM2.5,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
15,Nice Ouest Botanique,Ozone,118,100,109,121,119,109,111,96,...,147,140,128,116,98,103,81,79,74,68
18,Aéroport de Nice,Ozone,58,58,69,77,78,77,80,60,...,101,99,97,84,67,71,75,72,65,61
20,Aéroport de Nice,Particules PM2.5,-,-,-,-,-,-,-,-,...,17,13,15,11,14,10,9,11,9,8
21,Nice Arson,Ozone,58,63,62,66,70,76,66,58,...,103,108,107,91,80,74,59,54,43,37


In [62]:
parca_table=parca_table.melt(id_vars=["Station",'Mesure'],var_name="datetime")

In [63]:
parca_table["value"] = list(map( lambda value: None if value == "-" else int(value), parca_table.value))

In [64]:
parca_table.columns=Index(['station', 'measure', 'datetime', 'value'], dtype='object')

In [65]:
parca_table.head()

,station,measure,datetime,value
0,Nice_Port,Particules PM2.5,08/08/2017 0:00,NaN
1,Nice Ouest Botanique,Ozone,08/08/2017 0:00,118.0
2,Aéroport de Nice,Ozone,08/08/2017 0:00,58.0
3,Aéroport de Nice,Particules PM2.5,08/08/2017 0:00,NaN
4,Nice Arson,Ozone,08/08/2017 0:00,58.0


In [66]:
date_time=list(map(lambda string: datetime.strptime(string.replace('.1',''),'%d/%m/%Y %H:%M'),parca_table['datetime']))

In [67]:
parca_table['datetime']=date_time 

In [70]:
parca_table=parca_table.groupby(["datetime","station","measure"]).max().reset_index()

In [72]:
parca_table.to_csv("../stage/parca_data.csv",index=False)